## Read Demographic dataset 

In [2]:
import pandas as pd

# Read csv file save it in dataframe
demographic_covid_df = pd.read_csv("../../data/Demographic.csv")

# save dataframe to csv file and remove headers like "DP_04M"
demographic_covid_df.to_csv('Demographic.csv', header=False, index=False)

FileNotFoundError: [Errno 2] File b'../../data/Demographic.csv' does not exist: b'../../data/Demographic.csv'

## Display Demographic Dataset

In [ ]:
# Read csv file without header and save it in dataframe
demographic_df = pd.read_csv("Demographic.csv")
demographic_df.head(2)

# Drop unwanted columns

In [ ]:
# Remove columns which shows margin of errors, percent estimate and percent margin of error 

demographic_df = demographic_df[demographic_df.columns.drop(list(demographic_df.filter(regex='Margin')))]
demographic_df = demographic_df[demographic_df.columns.drop(list(demographic_df.filter(regex='Percent')))]

## Split id and Geographica Area Name

In [ ]:
#Select column "id" from Demographic_df, and choose a substring of each value in the column which contains county FIPS,
#and append it to a list   

county_FIPS_list = []
for i in range(len(demographic_df["id"])):
    county_FIPS_list.append(demographic_df.loc[i,'id'][9:])
    
#Add the list as a new column to the Demographic_df     
demographic_df["countyFIPS"] = county_FIPS_list

#Convert the type of countyFIPS from string to int
demographic_df['countyFIPS'] = demographic_df['countyFIPS'].astype(int)


In [ ]:
# Split geographical area name into county and state name

demographic_df[['County Name','State']] = demographic_df["Geographic Area Name"].str.split(",",expand=True) 

In [ ]:
demographic_df.head(2)

## In order to calcute correlation, read the confirmed cases and death datasets, and calculate the total number of confirmed cases and death for each county, merge them with Demographic dataset

In [ ]:
# Read csv confirmed cases file and save it in a dataframe
confirmed_df = pd.read_csv("../../data/COVID-19 Dataset/confirmed.csv")

# Read csv death file and save it in a dataframe
death_df = pd.read_csv("../../data/COVID-19 Dataset/deaths.csv")

In [ ]:
#sum up confirmed cases for each county
confirmed_df['Confirmed Total']= confirmed_df.iloc[:, 4:].sum(axis=1)

#sum up death for each county
death_df['death Total']= death_df.iloc[:, 4:].sum(axis=1)

In [ ]:
# define a new dataframe for confirmed cases which contains "countyFIPS", "County Name", "Confirmed Total"
modified_confirmed_df = confirmed_df[["countyFIPS", "County Name","State", "Confirmed Total"]]

# define a new dataframe for death cases which contains "countyFIPS", "County Name", "death Total"
modified_death_df = death_df[["countyFIPS", "County Name", "State","death Total"]]

In [ ]:
#merge modified dataframes based on "countyFIPS", "County Name"
modified_confirmed_death_df = pd.merge(modified_confirmed_df,modified_death_df, on = ["countyFIPS", "County Name" ,"State"])

modified_confirmed_death_df.head()

## Merge modified_confirmed_death_df with demographic dataset and then calculate correlation of each column with total confirmed and total death of each county

In [ ]:
# merge demographic df with modified_confirmed_death
covid_demographic_ConfirmedDeath_df = pd.merge(modified_confirmed_death_df,demographic_df, on = ["countyFIPS"])
covid_demographic_ConfirmedDeath_df

In [ ]:
# define a list which contains column headers of covid_demographic_ConfirmedDeath_df
columns = covid_demographic_ConfirmedDeath_df.columns
columns_modified = columns[8:-2]


In [ ]:
# calculate correlation for each column with total Confirmed cases and then append it to a list
correlation_confirmed = []
for i in range(len(columns_modified)-1):
    corr = covid_demographic_ConfirmedDeath_df["Confirmed Total"].corr(covid_demographic_ConfirmedDeath_df[columns_modified[i]])
    correlation_confirmed.append([columns_modified[i], corr])

#Sort the list 
correlation_confirmed = sorted(correlation_confirmed, key=lambda x: -x[1])
correlation_confirmed[:15]

In [ ]:
# calculate correlation for each column with total death and then append it to a list
correlation_death = []
for i in range(len(columns_modified)-1):
    corr = covid_demographic_ConfirmedDeath_df["death Total"].corr(covid_demographic_ConfirmedDeath_df[columns_modified[i]])
    correlation_death.append([columns_modified[i], corr])
    
#Sort the list 
correlation_death = sorted(correlation_death, key=lambda x: -x[1])
correlation_death[:15]

## Choose important columns based on correlation values and define a new dataframe

In [ ]:
# sum up "under 5" and "5 to 9"
demographic_df["Estimate!!SEX AND AGE!!Total population!!Under 10 years"]= demographic_df.iloc[:, 64:66].sum(axis=1)

# sum up "10 to 14" and "15 to 19"
demographic_df["Estimate!!SEX AND AGE!!Total population!!10 to 19 years"] = demographic_df.iloc[:, 66:68].sum(axis=1)

# sum up "20 to 24" and "25 to 34"
demographic_df["Estimate!!SEX AND AGE!!Total population!!20 to 34 years"] = demographic_df.iloc[:, 68:70].sum(axis=1)

# sum up "55 to 59" and "60 to 64"
demographic_df["Estimate!!SEX AND AGE!!Total population!!55 to 64 years"] = demographic_df.iloc[:, 72:74].sum(axis=1)

In [ ]:
demographic_df.head(2)

In [ ]:
demographic_df_final = demographic_df[["countyFIPS", "Estimate!!SEX AND AGE!!Total population!!Under 10 years",
                                     "Estimate!!SEX AND AGE!!Total population!!10 to 19 years", "Estimate!!SEX AND AGE!!Total population!!20 to 34 years",
                                     "Estimate!!SEX AND AGE!!Total population!!35 to 44 years", "Estimate!!SEX AND AGE!!Total population!!45 to 54 years",
                                     "Estimate!!SEX AND AGE!!Total population!!55 to 64 years", "Estimate!!SEX AND AGE!!Total population!!65 to 74 years",
                                     "Estimate!!SEX AND AGE!!Total population!!75 to 84 years", "Estimate!!SEX AND AGE!!Total population!!85 years and over",
                                     "Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female", "Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male",
                                     "Estimate!!SEX AND AGE!!Total population!!Female","Estimate!!SEX AND AGE!!Total population!!Male",
                                      "Estimate!!RACE!!Total population!!One race!!White","Estimate!!RACE!!Total population!!One race!!Black or African American",
                                     "Estimate!!HISPANIC OR LATINO AND RACE!!Total population", "Estimate!!RACE!!Total population!!One race!!Asian"]]

In [ ]:
demographic_df_final

## Read the main covid dataset and merge it with demographic_df_final

In [ ]:
#read the covid dataset
covid_df = pd.read_csv("../../data/output/covid.csv")

In [ ]:
#merge covid_df with demographic_df_final 
covid_demoghraphic_df = pd.merge(covid_df,demographic_df_final, on = "countyFIPS", how = "left")
covid_demoghraphic_df = covid_demoghraphic_df.drop(["Unnamed: 0"], axis = 1)

#display final dataframe
covid_demoghraphic_df.head()

In [ ]:
# save the final datafram in csv file
covid_demoghraphic_df.to_csv("../../data/output/COVID19_DEMOGRAPHIC_MERGE.csv",index=False)